In [5]:
import os
import torch
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms, models, datasets
import torch.optim as optim
import torch.nn as nn
from sklearn.model_selection import KFold
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, precision_recall_fscore_support)
from transformers import ViTForImageClassification, ViTFeatureExtractor
import optuna

In [6]:
dataset_dir ='/kaggle/input/deepfake/DeepFake'

In [7]:
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),  
    transforms.ToTensor(),  
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(p=0.2),
    transforms.RandomRotation(15),
    transforms.RandomCrop(224, padding=10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomAffine(degrees=20, scale=(0.8, 1.2), shear=10),
    transforms.RandomErasing(p=0.3),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  
])

transform_val_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])


In [8]:
# Load the dataset
from torchvision.datasets import ImageFolder
dataset = ImageFolder(root=dataset_dir, transform=transform_train)
print("Classes:", dataset.classes)
print("Class-to-Index Mapping:", dataset.class_to_idx)
print("Number of Samples:", len(dataset))

Classes: ['Fake', 'Real']
Class-to-Index Mapping: {'Fake': 0, 'Real': 1}
Number of Samples: 10826


In [6]:
def get_model(model_name):
    if model_name == "vgg-16":
        model = models.vgg16(pretrained=True)
        
        # Freeze the convolutional layers
        for param in model.features.parameters():
            param.requires_grad = False
        
        # Update the fully connected layer
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, 2)  # Change the last layer to output 2 classes
        return model

In [9]:
# Calculate metrics function
def calculate_metrics(model, loader, device):
    
    # Set the model to evaluation mode (disables dropout)
    model.eval()

    # Lists to store true labels and predicted labels
    all_labels = []
    all_predictions = []

    # Disabling gradient computation
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
             # Get predicted labels by taking the argmax (most likely class)
            _, predicted = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

     # Calculate the confusion matrix,which give TN, FP, FN, and TP
    conf_matrix = confusion_matrix(all_labels, all_predictions)
    # Unpack the confusion matrix into four components: TN, FP, FN, TP
    TN, FP, FN, TP = conf_matrix.ravel() 

    total = conf_matrix.sum()
    accuracy = (TP + TN) / total if total > 0 else 0.0
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0.0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0.0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0
    
    return accuracy, precision, recall, f1, conf_matrix


In [10]:
# Train the model function with validation accuracy printed after each epoch
def train_model(model, train_loader, val_loader, optimizer, criterion, device, epochs=5):
    # Variable to track the best validation accuracy
    best_val_accuracy = 0
    
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
         # Iterate over batches in the training data
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(train_loader)}")
        # Validation phase
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        val_accuracy = 100 * correct / total
        print(f"Epoch {epoch+1}/{epochs}, Validation Accuracy: {val_accuracy:.2f}%")
        
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
    
    return best_val_accuracy

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Cross-validation setup
num_folds = 3
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)


In [9]:
def objective(trial, model_name):
    # Get a suggested learning rate from Optuna
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
    
    # Initialize the model with dropout
    model = get_model(model_name).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    val_accuracies = []
    for fold_idx, (train_val_idx, test_idx) in enumerate(kf.split(dataset)):
        print(f"Fold {fold_idx + 1}/{num_folds}")
        
        # Create training/validation split
        train_val_data = Subset(dataset, train_val_idx)
        test_data = Subset(dataset, test_idx)
        
        train_size = int(0.8 * len(train_val_data))
        val_size = len(train_val_data) - train_size
        train_data, val_data = torch.utils.data.random_split(
            train_val_data, [train_size, val_size], generator=torch.Generator().manual_seed(42)
        )
        train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
        
        # Train the model and get validation accuracy
        train_model(model, train_loader, val_loader, optimizer, criterion, device, epochs=5)
        
        # Evaluate on validation set
        val_accuracy, _, _, _, _ = calculate_metrics(model, val_loader, device)
        val_accuracies.append(val_accuracy)
    
    # Return the average validation accuracy across all folds as the objective value
    return np.mean(val_accuracies)


In [11]:
def evaluate_test_set(model_name, best_lr):
    # Initialize model with the best learning rate
    model = get_model(model_name).to(device)
    optimizer = optim.Adam(model.parameters(), lr=best_lr)
    criterion = nn.CrossEntropyLoss()

    fold_metrics = []
    for fold_idx, (train_val_idx, test_idx) in enumerate(kf.split(dataset)):
        print(f"\nEvaluating on Fold {fold_idx + 1}/{num_folds}")
        
        # Create training/validation split
        train_val_data = Subset(dataset, train_val_idx)
        test_data = Subset(dataset, test_idx)
        
        train_size = int(0.8 * len(train_val_data))
        val_size = len(train_val_data) - train_size
        train_data, val_data = torch.utils.data.random_split(
            train_val_data, [train_size, val_size], generator=torch.Generator().manual_seed(42)
        )
        train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
        
        # Train the model
        train_model(model, train_loader, val_loader, optimizer, criterion, device, epochs=5)
        
        # Evaluate on the test set
        test_loader = DataLoader(test_data, batch_size=32, shuffle=False)
        fold_metrics.append(calculate_metrics(model, test_loader, device))
    
    # Print metrics for each fold
    for fold_idx, metrics in enumerate(fold_metrics):
        accuracy, precision, recall, f1, conf_matrix = metrics
        print(f"Fold {fold_idx + 1} Metrics:")
        print(f"Accuracy: {accuracy:.2f}, Precision: {precision:.2f}, Recall: {recall:.2f}, F1-Score: {f1:.2f}")
        print(f"Confusion Matrix:\n{conf_matrix}")

# Calculate average metrics across folds
    avg_accuracy = np.mean([metrics[0] for metrics in fold_metrics])
    avg_precision = np.mean([metrics[1] for metrics in fold_metrics])
    avg_recall = np.mean([metrics[2] for metrics in fold_metrics])
    avg_f1 = np.mean([metrics[3] for metrics in fold_metrics])
    total_conf_matrix = np.sum([metrics[4] for metrics in fold_metrics], axis=0)

    print("\nAverage Metrics Across Folds:")
    print(f"Accuracy: {avg_accuracy:.2f}, Precision: {avg_precision:.2f}, Recall: {avg_recall:.2f}, F1-Score: {avg_f1:.2f}")
    print(f"Confusion Matrix (sum of all folds):\n{total_conf_matrix}")


# Optuna Optimization and Final Testing
for model_name in [ "vgg-16"]:
    print(f"\nOptimizing for {model_name.upper()}...")
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: objective(trial, model_name), n_trials=5)  # You can increase the number of trials if needed

    # Best learning rate found for the model
    best_lr = study.best_params['lr']
    print(f"Best Learning Rate for {model_name.upper()}: {best_lr}")

    # Evaluate on test sets for each fold
    evaluate_test_set(model_name, best_lr)
    


[I 2024-12-24 23:31:29,649] A new study created in memory with name: no-name-c1a28ac1-39ce-45cd-b81c-9f92ab264b9f



Optimizing for VGG-16...


<ipython-input-9-2e749100ab16>:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hu

Fold 1/3
Epoch 1/5, Loss: 0.6363204455836702
Epoch 1/5, Validation Accuracy: 64.13%
Epoch 2/5, Loss: 0.5689530168449023
Epoch 2/5, Validation Accuracy: 69.74%
Epoch 3/5, Loss: 0.5611183839278985
Epoch 3/5, Validation Accuracy: 73.34%
Epoch 4/5, Loss: 0.5333395475182086
Epoch 4/5, Validation Accuracy: 67.87%
Epoch 5/5, Loss: 0.5253075162020836
Epoch 5/5, Validation Accuracy: 72.78%
Fold 2/3
Epoch 1/5, Loss: 0.5215318700226631
Epoch 1/5, Validation Accuracy: 75.28%
Epoch 2/5, Loss: 0.516530220844469
Epoch 2/5, Validation Accuracy: 70.91%
Epoch 3/5, Loss: 0.5123764760586438
Epoch 3/5, Validation Accuracy: 74.79%
Epoch 4/5, Loss: 0.5014847739296064
Epoch 4/5, Validation Accuracy: 76.11%
Epoch 5/5, Loss: 0.4950379110172967
Epoch 5/5, Validation Accuracy: 76.04%
Fold 3/3
Epoch 1/5, Loss: 0.4976999944086233
Epoch 1/5, Validation Accuracy: 75.28%
Epoch 2/5, Loss: 0.4962606650689689
Epoch 2/5, Validation Accuracy: 77.15%
Epoch 3/5, Loss: 0.4886985241708176
Epoch 3/5, Validation Accuracy: 75.62%

[I 2024-12-25 00:06:48,226] Trial 0 finished with value: 0.76061865189289 and parameters: {'lr': 0.00023232414378777328}. Best is trial 0 with value: 0.76061865189289.
<ipython-input-9-2e749100ab16>:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_

...

Fold 1/3
Epoch 1/5, Loss: 0.6318017502845321
Epoch 1/5, Validation Accuracy: 67.04%
Epoch 2/5, Loss: 0.5685112212902933
Epoch 2/5, Validation Accuracy: 71.54%
Epoch 3/5, Loss: 0.5407860717062134
Epoch 3/5, Validation Accuracy: 71.26%
Epoch 4/5, Loss: 0.5341001719401027
Epoch 4/5, Validation Accuracy: 71.47%
Epoch 5/5, Loss: 0.5236142284303739
Epoch 5/5, Validation Accuracy: 73.34%
Fold 2/3
Epoch 1/5, Loss: 0.5265970475436574
Epoch 1/5, Validation Accuracy: 74.24%
Epoch 2/5, Loss: 0.5174212459042586
Epoch 2/5, Validation Accuracy: 75.21%
Epoch 3/5, Loss: 0.49024172596509946
Epoch 3/5, Validation Accuracy: 76.11%
Epoch 4/5, Loss: 0.5012418575049764
Epoch 4/5, Validation Accuracy: 77.49%
Epoch 5/5, Loss: 0.49550394326942404
Epoch 5/5, Validation Accuracy: 75.07%
Fold 3/3
Epoch 1/5, Loss: 0.4893156334510824
Epoch 1/5, Validation Accuracy: 77.08%
Epoch 2/5, Loss: 0.49061598517618127
Epoch 2/5, Validation Accuracy: 73.13%
Epoch 3/5, Loss: 0.4954683405259696
Epoch 3/5, Validation Accuracy: 78

[I 2024-12-25 01:50:24,682] Trial 3 finished with value: 0.7479224376731302 and parameters: {'lr': 0.00018454251688395138}. Best is trial 0 with value: 0.76061865189289.
<ipython-input-9-2e749100ab16>:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG1

Fold 1/3
Epoch 1/5, Loss: 0.7465211078936224
Epoch 1/5, Validation Accuracy: 67.24%
Epoch 2/5, Loss: 0.7070373435046792
Epoch 2/5, Validation Accuracy: 54.85%
Epoch 3/5, Loss: 0.6848726182023465
Epoch 3/5, Validation Accuracy: 69.88%
Epoch 4/5, Loss: 0.6488507241833934
Epoch 4/5, Validation Accuracy: 72.44%
Epoch 5/5, Loss: 0.6311506630636711
Epoch 5/5, Validation Accuracy: 69.46%
Fold 2/3
Epoch 1/5, Loss: 0.6180676541275741
Epoch 1/5, Validation Accuracy: 72.58%
Epoch 2/5, Loss: 0.6055443578332828
Epoch 2/5, Validation Accuracy: 74.52%
Epoch 3/5, Loss: 0.6045829059340019
Epoch 3/5, Validation Accuracy: 73.13%
Epoch 4/5, Loss: 0.6219024468851353
Epoch 4/5, Validation Accuracy: 71.81%
Epoch 5/5, Loss: 0.6032846448171204
Epoch 5/5, Validation Accuracy: 73.68%
Fold 3/3
Epoch 1/5, Loss: 0.6100661316629272
Epoch 1/5, Validation Accuracy: 74.17%
Epoch 2/5, Loss: 0.5856962656777208
Epoch 2/5, Validation Accuracy: 73.34%
Epoch 3/5, Loss: 0.5889473035519953
Epoch 3/5, Validation Accuracy: 75.00

[I 2024-12-25 02:24:54,164] Trial 4 finished with value: 0.7326869806094183 and parameters: {'lr': 0.0009801144162306348}. Best is trial 0 with value: 0.76061865189289.
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Best Learning Rate for VGG-16: 0.00023232414378777328

Evaluating on Fold 1/3
Epoch 1/5, Loss: 0.6319872705646641
Epoch 1/5, Validation Accuracy: 68.49%
Epoch 2/5, Loss: 0.5780117880573589
Epoch 2/5, Validation Accuracy: 72.51%
Epoch 3/5, Loss: 0.5434516859318012
Epoch 3/5, Validation Accuracy: 72.44%
Epoch 4/5, Loss: 0.5358109084776094
Epoch 4/5, Validation Accuracy: 69.53%
Epoch 5/5, Loss: 0.5218565422856347
Epoch 5/5, Validation Accuracy: 71.54%

Evaluating on Fold 2/3
Epoch 1/5, Loss: 0.523648278653951
Epoch 1/5, Validation Accuracy: 75.83%
Epoch 2/5, Loss: 0.513523651914702
Epoch 2/5, Validation Accuracy: 75.83%
Epoch 3/5, Loss: 0.5125458039631501
Epoch 3/5, Validation Accuracy: 77.08%
Epoch 4/5, Loss: 0.4979021134297492
Epoch 4/5, Validation Accuracy: 73.48%
Epoch 5/5, Loss: 0.489788967107541
Epoch 5/5, Validation Accuracy: 75.35%

Evaluating on Fold 3/3
Epoch 1/5, Loss: 0.5019226293208191
Epoch 1/5, Validation Accuracy: 75.69%
Epoch 2/5, Loss: 0.5059820127092014
Epoch 2/5, Valid

In [11]:
import torch
import torch.nn as nn
from torchvision import models

def initialize_model(name):
    if name == "vgg-16":
        model = models.vgg16(pretrained=True)

        # Freeze all layers initially
        for param in model.parameters():
            param.requires_grad = False

        # Unfreeze the last convolutional block
        for param in model.features[24:].parameters():
            param.requires_grad = True

        # Modify the classifier for binary classification
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, 2)

        # Print trainable parameters
        def count_trainable_params(model):
            return sum(p.numel() for p in model.parameters() if p.requires_grad)

        print(f"Total trainable parameters: {count_trainable_params(model):,}")

    else:
        raise ValueError("Model name must be 'vgg16'")

    return model


In [12]:
# for fine tunning
def objective(trial, model_name):
    # Get a suggested learning rate from Optuna
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
    
    # Initialize the model with dropout
    model = initialize_model(model_name).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    val_accuracies = []
    for fold_idx, (train_val_idx, test_idx) in enumerate(kf.split(dataset)):
        print(f"Fold {fold_idx + 1}/{num_folds}")
        
        # Create training/validation split
        train_val_data = Subset(dataset, train_val_idx)
        test_data = Subset(dataset, test_idx)
        
        train_size = int(0.8 * len(train_val_data))
        val_size = len(train_val_data) - train_size
        train_data, val_data = torch.utils.data.random_split(
            train_val_data, [train_size, val_size], generator=torch.Generator().manual_seed(42)
        )
        train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
        
        # Train the model and get validation accuracy
        train_model(model, train_loader, val_loader, optimizer, criterion, device, epochs=5)
        
        # Evaluate on validation set
        val_accuracy, _, _, _, _ = calculate_metrics(model, val_loader, device)
        val_accuracies.append(val_accuracy)
    
    # Return the average validation accuracy across all folds as the objective value
    return np.mean(val_accuracies)


In [13]:
def evaluate_test_set(model_name, best_lr):
    # Initialize model with the best learning rate
    model = initialize_model(model_name).to(device)
    optimizer = optim.Adam(model.parameters(), lr=best_lr)
    criterion = nn.CrossEntropyLoss()

    fold_metrics = []
    for fold_idx, (train_val_idx, test_idx) in enumerate(kf.split(dataset)):
        print(f"\nEvaluating on Fold {fold_idx + 1}/{num_folds}")
        
        # Create training/validation split
        train_val_data = Subset(dataset, train_val_idx)
        test_data = Subset(dataset, test_idx)
        
        train_size = int(0.8 * len(train_val_data))
        val_size = len(train_val_data) - train_size
        train_data, val_data = torch.utils.data.random_split(
            train_val_data, [train_size, val_size], generator=torch.Generator().manual_seed(42)
        )
        train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_data, batch_size=32, shuffle=False)
        
        # Train the model
        train_model(model, train_loader, val_loader, optimizer, criterion, device, epochs=5)
        
        # Evaluate on the test set
        test_loader = DataLoader(test_data, batch_size=32, shuffle=False)
        fold_metrics.append(calculate_metrics(model, test_loader, device))
    
    # Print metrics for each fold
    for fold_idx, metrics in enumerate(fold_metrics):
        accuracy, precision, recall, f1, conf_matrix = metrics
        print(f"Fold {fold_idx + 1} Metrics:")
        print(f"Accuracy: {accuracy:.2f}, Precision: {precision:.2f}, Recall: {recall:.2f}, F1-Score: {f1:.2f}")
        print(f"Confusion Matrix:\n{conf_matrix}")

# Calculate average metrics across folds
    avg_accuracy = np.mean([metrics[0] for metrics in fold_metrics])
    avg_precision = np.mean([metrics[1] for metrics in fold_metrics])
    avg_recall = np.mean([metrics[2] for metrics in fold_metrics])
    avg_f1 = np.mean([metrics[3] for metrics in fold_metrics])
    total_conf_matrix = np.sum([metrics[4] for metrics in fold_metrics], axis=0)

    print("\nAverage Metrics Across Folds:")
    print(f"Accuracy: {avg_accuracy:.2f}, Precision: {avg_precision:.2f}, Recall: {avg_recall:.2f}, F1-Score: {avg_f1:.2f}")
    print(f"Confusion Matrix (sum of all folds):\n{total_conf_matrix}")


# Optuna Optimization and Final Testing
for model_name in [ "vgg-16"]:
    print(f"\nOptimizing for {model_name.upper()}...")
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: objective(trial, model_name), n_trials=5)  # You can increase the number of trials if needed

    # Best learning rate found for the model
    best_lr = study.best_params['lr']
    print(f"Best Learning Rate for {model_name.upper()}: {best_lr}")

    # Evaluate on test sets for each fold
    evaluate_test_set(model_name, best_lr)
    


[I 2024-12-25 13:36:47,042] A new study created in memory with name: no-name-17b0a0f1-9a96-4029-a6ca-5a8d6ab07448



Optimizing for VGG-16...


<ipython-input-12-1c309b109a5c>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/h

Total trainable parameters: 7,087,618
Fold 1/3
Epoch 1/5, Loss: 0.5766741253065141
Epoch 1/5, Validation Accuracy: 74.58%
Epoch 2/5, Loss: 0.47229872503991943
Epoch 2/5, Validation Accuracy: 75.14%
Epoch 3/5, Loss: 0.436988354881824
Epoch 3/5, Validation Accuracy: 78.32%
Epoch 4/5, Loss: 0.4024952099303514
Epoch 4/5, Validation Accuracy: 81.58%
Epoch 5/5, Loss: 0.3973448743313057
Epoch 5/5, Validation Accuracy: 82.69%
Fold 2/3
Epoch 1/5, Loss: 0.3938990489718664
Epoch 1/5, Validation Accuracy: 83.24%
Epoch 2/5, Loss: 0.3623178517291559
Epoch 2/5, Validation Accuracy: 82.55%
Epoch 3/5, Loss: 0.3472687445622123
Epoch 3/5, Validation Accuracy: 84.49%
Epoch 4/5, Loss: 0.33660881986934177
Epoch 4/5, Validation Accuracy: 84.35%
Epoch 5/5, Loss: 0.3235711974827624
Epoch 5/5, Validation Accuracy: 83.31%
Fold 3/3
Epoch 1/5, Loss: 0.34186439623654874
Epoch 1/5, Validation Accuracy: 85.04%
Epoch 2/5, Loss: 0.31723081431665473
Epoch 2/5, Validation Accuracy: 85.18%
Epoch 3/5, Loss: 0.3045534504711

[I 2024-12-25 14:13:50,200] Trial 0 finished with value: 0.8331024930747922 and parameters: {'lr': 0.00010837400852480871}. Best is trial 0 with value: 0.8331024930747922.
<ipython-input-12-1c309b109a5c>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=V

...

Total trainable parameters: 7,087,618
Fold 1/3
Epoch 1/5, Loss: 1.125412908542222
Epoch 1/5, Validation Accuracy: 63.78%
Epoch 2/5, Loss: 0.6066299257028168
Epoch 2/5, Validation Accuracy: 74.79%
Epoch 3/5, Loss: 0.5465516787207587
Epoch 3/5, Validation Accuracy: 76.39%
Epoch 4/5, Loss: 0.49954844887744
Epoch 4/5, Validation Accuracy: 75.00%
Epoch 5/5, Loss: 0.48699353643543813
Epoch 5/5, Validation Accuracy: 78.88%
Fold 2/3
Epoch 1/5, Loss: 0.4640914455303171
Epoch 1/5, Validation Accuracy: 78.95%
Epoch 2/5, Loss: 0.44638792676490974
Epoch 2/5, Validation Accuracy: 79.99%
Epoch 3/5, Loss: 0.44498560178345736
Epoch 3/5, Validation Accuracy: 79.29%
Epoch 4/5, Loss: 0.4389144851522551
Epoch 4/5, Validation Accuracy: 81.23%
Epoch 5/5, Loss: 0.46077225995327226
Epoch 5/5, Validation Accuracy: 65.24%
Fold 3/3
Epoch 1/5, Loss: 0.4784979752743442
Epoch 1/5, Validation Accuracy: 80.26%
Epoch 2/5, Loss: 0.4096433665212347
Epoch 2/5, Validation Accuracy: 80.19%
Epoch 3/5, Loss: 0.416691598230304

[I 2024-12-25 16:02:30,631] Trial 3 finished with value: 0.7469990766389657 and parameters: {'lr': 0.0033356263825548888}. Best is trial 0 with value: 0.8331024930747922.
<ipython-input-12-1c309b109a5c>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-5, 1e-1)
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VG

Total trainable parameters: 7,087,618
Fold 1/3
Epoch 1/5, Loss: 8.163281998252343
Epoch 1/5, Validation Accuracy: 51.39%
Epoch 2/5, Loss: 1.890694885293423
Epoch 2/5, Validation Accuracy: 49.38%
Epoch 3/5, Loss: 0.7053680736057008
Epoch 3/5, Validation Accuracy: 50.48%
Epoch 4/5, Loss: 0.700878318172792
Epoch 4/5, Validation Accuracy: 49.38%
Epoch 5/5, Loss: 0.6984701215891548
Epoch 5/5, Validation Accuracy: 50.62%
Fold 2/3
Epoch 1/5, Loss: 0.6996491597502271
Epoch 1/5, Validation Accuracy: 50.97%
Epoch 2/5, Loss: 0.694748653232722
Epoch 2/5, Validation Accuracy: 50.00%
Epoch 3/5, Loss: 0.6983968378430572
Epoch 3/5, Validation Accuracy: 50.28%
Epoch 4/5, Loss: 0.6976875926249594
Epoch 4/5, Validation Accuracy: 50.00%
Epoch 5/5, Loss: 0.6988420598414722
Epoch 5/5, Validation Accuracy: 50.35%
Fold 3/3
Epoch 1/5, Loss: 0.6965940436605591
Epoch 1/5, Validation Accuracy: 48.41%
Epoch 2/5, Loss: 0.695357051343549
Epoch 2/5, Validation Accuracy: 51.59%
Epoch 3/5, Loss: 0.6983186352318822
Epoc

[I 2024-12-25 16:38:06,525] Trial 4 finished with value: 0.5083102493074793 and parameters: {'lr': 0.011046503969720027}. Best is trial 0 with value: 0.8331024930747922.
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Best Learning Rate for VGG-16: 0.00010837400852480871
Total trainable parameters: 7,087,618

Evaluating on Fold 1/3
Epoch 1/5, Loss: 0.5785066411310796
Epoch 1/5, Validation Accuracy: 75.62%
Epoch 2/5, Loss: 0.46792233994652555
Epoch 2/5, Validation Accuracy: 77.77%
Epoch 3/5, Loss: 0.4368656005154657
Epoch 3/5, Validation Accuracy: 77.63%
Epoch 4/5, Loss: 0.4045661822866998
Epoch 4/5, Validation Accuracy: 76.80%
Epoch 5/5, Loss: 0.3995853181701997
Epoch 5/5, Validation Accuracy: 81.65%

Evaluating on Fold 2/3
Epoch 1/5, Loss: 0.37996076335564505
Epoch 1/5, Validation Accuracy: 84.28%
Epoch 2/5, Loss: 0.3767143885569019
Epoch 2/5, Validation Accuracy: 84.56%
Epoch 3/5, Loss: 0.35357557368871256
Epoch 3/5, Validation Accuracy: 84.35%
Epoch 4/5, Loss: 0.34999272927065583
Epoch 4/5, Validation Accuracy: 82.96%
Epoch 5/5, Loss: 0.3334822432029972
Epoch 5/5, Validation Accuracy: 83.59%

Evaluating on Fold 3/3
Epoch 1/5, Loss: 0.33613505646668745
Epoch 1/5, Validation Accuracy: 84.63%
Epoch 